## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
from math import ceil

In [2]:
# Дані
p0 = 0.19
p1 = 0.20
alpha = 0.05

# Обчислення сумарної кількості користувачів для А/В-тесту
effect_size = sms.proportion_effectsize(p1, p0)       # Розрахунок розміру ефекту на основі наших очікуваних показників
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=alpha,
    ratio=1
    )                                                  # Розрахунок необхідного розміру вибірки для кожної групи

required_n = ceil(required_n)                          # Округлення до наступного цілого числа

print(f"Сумарна кількість користувачів для тесту складає {required_n * 2}.")

Сумарна кількість користувачів для тесту складає 49276.


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [3]:
# Завантаження даних
df = pd.read_csv('../data/statistical_hypothesis/cookie_cats.csv')

In [4]:
df[:6]

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
5,540,gate_40,187,True,True


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [6]:
# Перевірка унікальності користувачів
session_counts = df['userid'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'Є {multi_users} користувачів, які зустрічаються кілька разів у наборі даних.')

Є 0 користувачів, які зустрічаються кілька разів у наборі даних.


In [7]:
# Переконаємось щодо кількості користувачів в кожній групі
df['version'].value_counts()

version
gate_40    45489
gate_30    44700
Name: count, dtype: int64

In [8]:
# Середнє значення показника показника retention_7 по версіям гри
retention_rates = df.groupby('version')['retention_7'].mean()
retention_rates.round(4)

version
gate_30    0.1902
gate_40    0.1820
Name: retention_7, dtype: float64

Отримане значення відсотку утримання користувачів на сьомий день у контрольній групі (`gate_30`) відповідає заявленому продакт менеджером.  
Судячи з отриманих значень, для gate_40 відсоток утримання не збільшився (18.2% проти 19.02%).  
Вже видно, що попередня версія гри давала коращий результат, ніж нова версія.  
**Гіпотези:**  
$H_0: p_{treatment} = p_{control} $  (переміщення воріт не впливає на утримання)  
$H_a: p_{treatment} \ne p_{control} $ (відсотки утримання відрізняються)  
Тип тесту: двосторонній

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [11]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [12]:
control_results = df[df['version'] == 'gate_30']['retention_7']
treatment_results = df[df['version'] == 'gate_40']['retention_7']

In [13]:
# Загальна кількість користувачів в кожній групі
n_con = control_results.count()
n_treat = treatment_results.count()

# Формуємо масив успіхів та масив загальної кількості записів
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

In [14]:
# z-тест для пропорції у випадку двох вибірок
z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05, method='normal')

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.4f}')
print(f'Довірчий інтервал 95% для групи control (gate_30): [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Довірчий інтервал 95% для групи treatment (gate_40): [{lower_treat:.3f}, {upper_treat:.3f}]')

z statistic: 3.16
p-value: 0.0016
Довірчий інтервал 95% для групи control (gate_30): [0.187, 0.194]
Довірчий інтервал 95% для групи treatment (gate_40): [0.178, 0.186]


In [15]:
print('=' * 60)
print('ПРИЙНЯТТЯ РІШЕННЯ:')
print('=' * 60)
if pval < alpha:
    print(f"Оскільки {pval:.4f} < {alpha}. Відхиляємо гіпотезу H0.\nЦе означає, що переміщення воріт суттєво вплинуло на утримання користувачів.")
else:
    print("Не відхиляємо H0. Висновок: доказів недостатньо, щоб стверджувати, що переміщення воріт суттєво вплинуло на відсоток утримання користувачів.")

ПРИЙНЯТТЯ РІШЕННЯ:
Оскільки 0.0016 < 0.05. Відхиляємо гіпотезу H0.
Це означає, що переміщення воріт суттєво вплинуло на утримання користувачів.


**Висновки:**  
- контольна версія гри (gate_30) має краще утримання на 7й день, ніж тестова версія (gate_40);
- довірчі інтервали не перетинаються, це додатково підтверджує статистичну значущість різниці в поведінці користувачів.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


In [16]:
crosstab = pd.crosstab(df['version'], df['retention_7'])
crosstab

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


*Формулювання гіпотез для тесту Хі-квадрат:*
- Н0:
Версія гри не впливає на утримання гравця на 7-й день. Тобто, немає статистично значущої залежності між version і retention_7.
- Н1:
Версія гри впливає на утримання гравця. Є статистично значуща залежність між version і retention_7.


In [17]:
chi2, p, dof, expected = stats.chi2_contingency(crosstab)

print(f"χ² = {chi2:.3f}")
print(f"p-value = {p:.5f}")
print(f"Ступені свободи = {dof}")
print("Очікувані частоти:\n", expected)

χ² = 9.959
p-value = 0.00160
Ступені свободи = 1
Очікувані частоти:
 [[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]


In [18]:
print('=' * 60)
print('ПРИЙНЯТТЯ РІШЕННЯ:')
print('=' * 60)
if p < alpha:
    print(f"Оскільки {p:.4f} < {alpha}. Відхиляємо гіпотезу H0.\nЦе означає, що існує статистично значуща залежність між версією гри і тим, чи повернувся гравець на 7-й день.")
else:
    print("Не відхиляємо H0. Висновок: доказів недостатньо, щоб стверджувати, що версія гри впливає на утримання гравця на 7-й день.")

ПРИЙНЯТТЯ РІШЕННЯ:
Оскільки 0.0016 < 0.05. Відхиляємо гіпотезу H0.
Це означає, що існує статистично значуща залежність між версією гри і тим, чи повернувся гравець на 7-й день.
